In [2]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.applications import vgg19
from keras import backend as K

In [3]:
# Load and preprocess images
def load_and_preprocess_image(image_path, target_size=(100, 100)):
    img = load_img(image_path, target_size=target_size)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

In [4]:
# Convert a tensor to an image
def deprocess_image(x):
    x = x.reshape((x.shape[1], x.shape[2], 3))
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [5]:
content_layer = 'block5_conv2'
style_layers = [
    'block1_conv1',
    'block2_conv1',
    'block3_conv1',
    'block4_conv1',
    'block5_conv1',
]

In [6]:
def get_model():
  # Define the model
    vgg = vgg19.VGG19(weights='imagenet', include_top=False)
    vgg.trainable = False
    outputs = [vgg.get_layer(layer).output for layer in [content_layer] + style_layers]
    model = tf.keras.Model(inputs=vgg.input, outputs=outputs)
    return model


In [7]:
# Define the content loss
def content_loss(base, target):
    return tf.reduce_mean(tf.square(base - target))

# Define the style loss
def gram_matrix(x):
    channels = int(x.shape[-1])
    features = K.batch_flatten(K.reshape(x, (-1, channels)))
    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = tf.math.reduce_prod(tf.shape(combination))
    return tf.reduce_mean(tf.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

# Combine content and style losses
def total_loss(content_loss, style_loss, content_weight=1, style_weight=1e4):
    return content_weight * content_loss + style_weight * style_loss

# Define gradients and loss
def compute_loss(model, loss_weights, init_image, gram_style_features, content_features):
    model_outputs = model(init_image)
    style_output_features = model_outputs[1:]
    content_output_features = model_outputs[:1]

    style_score = 0
    content_score = 0

    weight_per_style_layer = 1.0 / float(len(style_layers))
    for target_style, comb_style in zip(gram_style_features, style_output_features):
        style_score += weight_per_style_layer * style_loss(comb_style[0], target_style)

    weight_per_content_layer = 1.0 / float(len(content_layer))
    content_score += weight_per_content_layer * content_loss(content_output_features[0], content_features[0])

    style_score *= loss_weights[0]
    content_score *= loss_weights[1]

    loss = style_score + content_score
    return loss, style_score, content_score

In [8]:
# Define the optimizer
def compute_grads(cfg):
    with tf.GradientTape() as tape:
        loss, _, _ = compute_loss(**cfg)
    grads = tape.gradient(loss, cfg['init_image'])
    return grads

In [9]:
# Define the content and style images
content_image_path = '../input/rickshaw-test-images/test.png'
style_image_path = '../input/rickshaw-test-images/rick.jpg'

content_image = load_and_preprocess_image(content_image_path)
style_image = load_and_preprocess_image(style_image_path)

In [10]:
# Style transfer function
def style_transfer(content_image, style_image, iterations=500, content_weight=1, style_weight=1e4):
    model = get_model()

    # Compute content and style features
    content_features = model(content_image)[:1]
    style_features = model(style_image)[1:]

    gram_style_features = [gram_matrix(style_feature[0]) for style_feature in style_features]

    # Initialize the generated image with the content image
    init_image = tf.Variable(content_image, dtype=tf.float32)

    # Define the optimizer
    opt = tf.optimizers.Adam(learning_rate=5, beta_1=0.99, epsilon=1e-1)

    # Define configuration
    loss_weights = (style_weight, content_weight)
    cfg = {
        'model': model,
        'loss_weights': loss_weights,
        'init_image': init_image,
        'gram_style_features': gram_style_features,
        'content_features': content_features
    }

    # Style transfer loop
    for i in range(iterations):
        grads = compute_grads(cfg)
        opt.apply_gradients([(grads, init_image)])
        clipped = tf.clip_by_value(init_image, 0.0, 255.0)
        init_image.assign(clipped)

        if i % 100 == 0:
            print(f"Iteration {i}")

    return deprocess_image(init_image.numpy())

In [11]:
# Run style transfer
target_size = (100, 100)  # Adjust the target size as needed
result = style_transfer(content_image, style_image, iterations=500, content_weight=1, style_weight=1e4)

# Display the result
import matplotlib.pyplot as plt
plt.imshow(result)
plt.show()

TypeError: Cannot convert 36.0 to EagerTensor of dtype int32